In [1]:
import pandas as pd
from lifelines import KaplanMeierFitter, CoxPHFitter
import matplotlib.pyplot as plt
import itertools as it

In [ ]:
# 'non-invasive', 'multifocal', 'tumour<1cm, grade 1', 'tumour>2cm, grade 2'
var = input()

In [ ]:
data = pd.read_csv("./data/testset_set_echte_data1_uitgebreid.csv", sep=';', on_bad_lines='skip')
data = data[data['F20DEATHCAU'] == "Kanker"]
data = data[data['F01INC07'] == 1]
data = data[data['RF01INC01'] == var]

In [ ]:
data.head()

In [ ]:
data['REGDAT'] = pd.to_datetime(data['REGDAT'], dayfirst=True, errors='coerce')
data['BIRTHDAT'] = pd.to_datetime(data['BIRTHDAT'], dayfirst=True, errors='coerce')


data['TIME'] = (data['REGDAT'] - data['BIRTHDAT']).dt.days // 365

In [ ]:
data.dtypes

In [ ]:
data.tail()




In [ ]:
time = "TIME"
event = "F20CRF"
method = "chemo"
method1 = "later"
method2 = "hormonth"
method3 = "neoadjther"
T = data[time]
E = data[event]
plt.hist(T, bins = 50)
plt.show()

In [ ]:
kmf = KaplanMeierFitter()
kmf.fit(durations = T, event_observed = E)
kmf.plot_survival_function()


In [ ]:
cph = CoxPHFitter()
columns = [time, event, method, method1, method2, method3]
cph.fit(data[columns], duration_col = time, event_col = event)
cph.print_summary()

In [ ]:
cph.plot()

In [ ]:
group_1 = pd.DataFrame({ method : [0], method1 : [0], method2: [0], method3: [0]})
group_2 = pd.DataFrame({ method : [1], method1 : [0], method2: [0], method3: [0]})
group_3 = pd.DataFrame({ method : [0], method1 : [1], method2: [0], method3: [0]})
group_4 = pd.DataFrame({ method : [0], method1 : [0], method2: [1], method3: [0]})
group_5 = pd.DataFrame({ method : [0], method1 : [0], method2: [0], method3: [1]})

ax = cph.predict_survival_function(group_1).plot(label="Chemo=0, Later=0, hormonth=0, neoadjther=0", linestyle='-', color="green")
cph.predict_survival_function(group_2).plot(ax=ax, label="Chemo=1, Later=0, hormonth=0, neoadjther=0", linestyle='-', color="yellow")
cph.predict_survival_function(group_3).plot(ax=ax, label="Chemo=0, Later=1, hormonth=0, neoadjther=0", linestyle='-', color="red")
cph.predict_survival_function(group_4).plot(ax=ax, label="Chemo=0, Later=0, hormonth=1, neoadjther=0", linestyle='-', color="blue")
cph.predict_survival_function(group_5).plot(ax=ax, label="Chemo=0, Later=0, hormonth=0, neoadjther=1", linestyle='-', color="black")


# Finalize plot
plt.title("Survival Curves for Different Treatment Groups")
plt.xlabel("Time")

plt.ylabel("Survival Probability")
plt.grid(True)
plt.legend()
plt.show()